Authors: Salvatore Mandra (salvatore.mandra@nasa.gov)<br>
&emsp;&emsp;&emsp;&emsp;Jeffrey Marshall (jeffrey.s.marshall@nasa.gov)

Copyright © 2021, United States Government, as represented by the Administrator
of the National Aeronautics and Space Administration. All rights reserved.

The *HybridQ: A Hybrid Simulator for Quantum Circuits* platform is licensed under
the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the License.

# XX - Noise

It is possible to add noise models in Kraus form to a Circuit in **HybridQ**. In this tutorial, we will show how to utilise default noise models already in the library, but also how the user can construct custom noise models. 

There are two relevant modules <code>hybridq.dm</code> and <code>hybridq.noise</code>, which allow one to construct a circuit with noise, and also provides some utility functions relvant to noisy simulations.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from hybridq.gate import Gate
from hybridq.circuit.simulation import simulate
from hybridq.extras.random import get_rqc

# density matrix imports
from hybridq.dm.circuit import Circuit as SuperCircuit
from hybridq.dm.gate import Gate as NoiseGate
from hybridq.dm.circuit.simulation import simulate as dm_simulate
from hybridq.noise.utils import add_depolarizing_noise
from hybridq.noise.channel.utils import ptrace, is_channel, is_dm, reconstruct_dm
from hybridq.noise.channel import *


## Custom Kraus maps
First we show how to build custom KrausGate's, either directly from the matrices themselves, or using pre-defined Gate's

In [2]:
# |0><0| and |1><1| projectors as a list
kraus_ops = [ [[1, 0], [0, 0]], [[0, 0], [0, 1]]]

# convert to Gate types acting on relevant qubit(s)
kraus_op_gates = [Gate('matrix', U=k, qubits=['my_qubit']) for k in kraus_ops]

kraus_gate = NoiseGate('kraus', gates=kraus_op_gates)

print(kraus_gate)

# check it defines a valid quantum channel
print(f'\nis_channel(kraus_gate) = {is_channel(kraus_gate)}\n')


KrausSuperGate(name='KRAUS', l_qubits=('my_qubit',), r_qubits=('my_qubit',), gates=(BaseTupleGate(MatrixGate(name='MATRIX', qubits=('my_qubit',), M=numpy.ndarray(shape=(2, 2), dtype=int64)), MatrixGate(name='MATRIX', qubits=('my_qubit',), M=numpy.ndarray(shape=(2, 2), dtype=int64))), BaseTupleGate(MatrixGate(name='MATRIX', qubits=('my_qubit',), M=numpy.ndarray(shape=(2, 2), dtype=int64)), MatrixGate(name='MATRIX', qubits=('my_qubit',), M=numpy.ndarray(shape=(2, 2), dtype=int64)))), s=1)

is_channel(kraus_gate) = True



We can also build more general map representation (generalizing the chi matrix): $\rho \rightarrow \sum_{i,j}s_{i,j} L_i \rho R_j^\dagger $

In [3]:
# note, this is just for demonstrative purposes, it is not a valid quantum map
print('first an invalid channel with left operators (X, Z), and right operators (I, Z)')
left_gates = [Gate('X', qubits=[0]), Gate('Z', qubits=[0])]
right_gates = [Gate('I', qubits=[0]), Gate('Y', qubits=[0])]

# specify custom left and right gates, and the 's' matrix
kraus_gate = NoiseGate('kraus', gates=[left_gates, right_gates], s=[[0.1, 0.2], [0.2, 0.9]])
print(kraus_gate)
print()

# as mentioned above, the above channel is not a valid quantum map
print(f'(X,Z), (I,Y) (invalid) channel: is_channel(kraus_gate) = {is_channel(kraus_gate)}\n')

# now let's build a valid channel
print('now a valid channel with left and right operators (X, Z)')
left_gates = [Gate('X', qubits=[0]), Gate('Z', qubits=[0])]
right_gates = [Gate('X', qubits=[0]), Gate('Z', qubits=[0])]

# specify custom left and right gates, and the 's' matrix
kraus_gate = NoiseGate('kraus', gates=[left_gates, right_gates], s=[[0.1, 0.2], [0.2, 0.9]])
print(kraus_gate)
print()

# as mentioned above, the above channel is not a valid quantum map
print(f'(X,Z), (X,Z) (valid) channel: is_channel(kraus_gate) = {is_channel(kraus_gate)}')

first an invalid channel with left operators (X, Z), and right operators (I, Z)
KrausSuperGate(name='KRAUS', l_qubits=(0,), r_qubits=(0,), gates=(BaseTupleGate(Gate_X(name='X', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Z(name='Z', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64))), BaseTupleGate(Gate_I(name='I', qubits=(0,)), Gate_Y(name='Y', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=complex128)))), s=numpy.ndarray(shape=(2, 2), dtype=float64))

(X,Z), (I,Y) (invalid) channel: is_channel(kraus_gate) = False

now a valid channel with left and right operators (X, Z)
KrausSuperGate(name='KRAUS', l_qubits=(0,), r_qubits=(0,), gates=(BaseTupleGate(Gate_X(name='X', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Z(name='Z', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64))), BaseTupleGate(Gate_X(name='X', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Z(name='Z', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=int64)))

## Named SuperGates

Here we show some of the Kraus maps which exist in the **HybridQ** library.
These are found in <code>hybridq.noise.channel</code>.
In addition to those shown below, there are also <code>GlobalPauliChannel</code> and <code>LocalPauliChannel</code> that implement Pauli channels (Kraus operators are Pauli operators).

In [4]:
# a depolarizing channel with probability of depolarizing `p`, acting on all qubits
gdc = GlobalDepolarizingChannel(qubits=[0,1,2], p=0.1)
print(gdc)

print()

# this is similar to above, except each qubit gets a single qubit depolarizing channel
# we can specify unique probabilities for each qubit
# when we print it, we see it returns a tuple of channels
ldc = LocalDepolarizingChannel(qubits=[0,1,2], p=[0.1, 0.2, 0.3])
print(ldc)

print()

# similar to LocalDepolarizingChannel, but applies a Pauli (e.g. Kraus ops are I and Z here)
dephase_channel = LocalDephasingChannel(qubits=[0,1,2], p=[0.1, 0.2, 0.3], pauli_index=3)
print(dephase_channel)

print()

# generalized amplitude damping channel (describing 0->1 and 1->0 incoherent transitions)
adc = AmplitudeDampingChannel(qubits=[0,1,2], gamma=0.1, p=0.9)
print(adc)

GlobalDepolarizingChannel(name='GLOBAL_DEPOLARIZING_CHANNEL', qubits=(0, 1, 2), s.shape=(64,), p=0.1)

(LocalDepolarizingChannel(name='LOCAL_DEPOLARIZING_CHANNEL', qubits=(0,), s.shape=(4,), p=0.1), LocalDepolarizingChannel(name='LOCAL_DEPOLARIZING_CHANNEL', qubits=(1,), s.shape=(4,), p=0.2), LocalDepolarizingChannel(name='LOCAL_DEPOLARIZING_CHANNEL', qubits=(2,), s.shape=(4,), p=0.3))

(LocalDephasingChannel(name='LOCAL_DEPHASING_CHANNEL', qubits=(0,), s.shape=(4,), p=0.1, axis=Z), LocalDephasingChannel(name='LOCAL_DEPHASING_CHANNEL', qubits=(1,), s.shape=(4,), p=0.2, axis=Z), LocalDephasingChannel(name='LOCAL_DEPHASING_CHANNEL', qubits=(2,), s.shape=(4,), p=0.3, axis=Z))

(AmplitudeDampingChannel(name='AMPLITUDE_DAMPING_CHANNEL', qubits=(0,), s.shape=(4,), gamma=0.1, p=0.9), AmplitudeDampingChannel(name='AMPLITUDE_DAMPING_CHANNEL', qubits=(1,), s.shape=(4,), gamma=0.1, p=0.9), AmplitudeDampingChannel(name='AMPLITUDE_DAMPING_CHANNEL', qubits=(2,), s.shape=(4,), gamma=0.1, p=0.9))


## Adding noise to a circuit

There are a few convenience functions that will add noise after or before the Gate's of a Circuit.

In addition to the depolarizing noise case shown below, currently there are also similar functions for dephasing noise <code>add_dephasing_noise</code> and amplitude damping noise <code>add_amplitude_damping_noise</code>, found in <code>hybridq.noise.utils</code>

In [5]:
random_circuit = get_rqc(3, 50)
print("random unitary circuit:")
print(random_circuit)
print()

# add 1% noise after single qubit gates, and 1.5% noise after two qubit gates
noisy_circuit = add_depolarizing_noise(random_circuit, probs=(0.01, 0.015))
print("circuit after adding noise:")
print(noisy_circuit)

/Users/jeff/opt/anaconda3/envs/hybridq/lib/python3.8/site-packages/hybridq/gate/gate.py:560: UserWarning: Using default number of qubits 'n_qubits=1'.
  warn("Using default number of qubits 'n_qubits=1'.")


random unitary circuit:
Circuit([
	Gate_SQRT_SWAP^*(name='SQRT_SWAP', qubits=(1, 2), M=numpy.ndarray(shape=(4, 4), dtype=complex128))**0.94448
	Gate_CZ^+(name='CZ', qubits=(0, 2), M=numpy.ndarray(shape=(4, 4), dtype=int64))**0.61783
	MatrixGate(name='MATRIX', qubits=(0,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	MatrixGate(name='MATRIX', qubits=(2, 1), M=numpy.ndarray(shape=(4, 4), dtype=complex128))
	...
	Gate_SQRT_Y(name='SQRT_Y', qubits=(2,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))**0.11861,
	MatrixGate(name='MATRIX', qubits=(2,), M=numpy.ndarray(shape=(2, 2), dtype=complex128)),
	MatrixGate(name='MATRIX', qubits=(1, 0), M=numpy.ndarray(shape=(4, 4), dtype=complex128)),
	Gate_ZZ^*(name='ZZ', qubits=(1, 0), M=numpy.ndarray(shape=(4, 4), dtype=int64))**0.05721
])

circuit after adding noise:
Circuit([
	Gate_SQRT_SWAP^*(name='SQRT_SWAP', qubits=(1, 2), M=numpy.ndarray(shape=(4, 4), dtype=complex128))**0.94448
	GlobalDepolarizingChannel(name='GLOBAL_DEPOLARIZING_CHANNE

## Perform density matrix simulation

In [6]:
nq = 2
random_circuit = get_rqc(nq, 25)
noisy_circuit = add_depolarizing_noise(random_circuit, probs=(0.01, 0.015))


# start in the + state, but can also provide np.ndarray as initial density matrix
# we reshape the output to be dxd (instead of split by tensor indices)
rho = np.reshape(dm_simulate(noisy_circuit, '+'), (2**nq, 2**nq))

print(rho)

# we can check it is a density matrix
print(f'output is_dm = {is_dm(rho)}')

/Users/jeff/opt/anaconda3/envs/hybridq/lib/python3.8/site-packages/hybridq/circuit/simulation/simulation.py:408: UserWarning: The system is too small to use optimize='evolution-hybridq'. Falling back to optimize='evolution-einsum'
  warn("The system is too small to use optimize='evolution-hybridq'. "


[[ 0.37538254-1.19558530e-09j -0.00285803-2.25299716e-01j
  -0.17299353+1.06610715e-01j  0.0218471 +1.98528036e-01j]
 [-0.00285807+2.25299746e-01j  0.2244091 -2.79082446e-08j
  -0.07323889-1.24401368e-01j -0.14706825+1.33162886e-02j]
 [-0.17299354-1.06610730e-01j -0.07323888+1.24401301e-01j
   0.20425305-2.52970489e-09j  0.0589449 -1.25093848e-01j]
 [ 0.02184708-1.98528081e-01j -0.14706822-1.33162662e-02j
   0.05894497+1.25093848e-01j  0.19595502-2.33144970e-08j]]
output is_dm = True


## different backends

All of the standard **HybridQ** backends are supported in density matrix simulations.

Here we show the tensor network backend

In [7]:
random_circuit = get_rqc(2, 25)
noisy_circuit = add_depolarizing_noise(random_circuit, probs=(0.01, 0.015))


# let's use the tensor network backed to trace out qubit 1
rho_0_tn = dm_simulate(noisy_circuit, initial_state='+', final_state='.a.a', optimize='tn')

# we can also get the same output using the ptrace function
rho = np.reshape(dm_simulate(noisy_circuit, '+', optimize='evolution-einsum'), (2**nq, 2**nq))
rho_0_full_evolution = ptrace(rho, keep=[0])

print('tensor network output:')
# round due to default precision (can increase using atol)
print(np.round(rho_0_tn, 6))

print('\nfull evolution with partial trace:')
print(np.round(rho_0_full_evolution, 6))

Contracting tensor (li=2^2, mli=2^26.0):   0%|          | 0/1 [00:00<?, ?it/s]

tensor network output:
[[0.537376-0.j       0.061301-0.315364j]
 [0.061301+0.315364j 0.462624+0.j      ]]

full evolution with partial trace:
[[0.537376+0.j       0.061301-0.315364j]
 [0.061301+0.315364j 0.462624-0.j      ]]


## Kraus operator pure state sampling

Instead of applying the Kraus operators directly on a density matrix, **HybridQ** supports an option to sample the Kraus operators probabilistically on a pure state. This can be done so long as the left and right Kraus operators are the same, i.e. of the form $\sum_i K_i \rho K_i^\dagger$. Then, for $\rho = |\psi \rangle \langle \psi |$, the Kraus operator $K_i$ is applied with probability $P_i = |\langle \psi | K_i^\dagger K_i |\psi\rangle|^2$.

To use sampling, the pure state simulation method <code>hybridq.circuit.simulation</code> should be used, specifying *allow_sampling=True*, which can accept a noise circuit <code>hybridq.dm.circuit</code> containing Kraus operations.

In [9]:
# First a unitary channel

# channel with the same left/right Kraus operators
# setting p=1 means I, X, Y, Z applied with equal probability
gdc = GlobalDepolarizingChannel([0], p=1.)
c = SuperCircuit([gdc])

# see example run on a single state
# Note, since this is a unitary channel, the Kraus operators are sampled before run-time.
# we set remove_id_gates=False so that when the identity is chosen the circuit still runs.
psi = simulate(c, '+', allow_sampling=True, remove_id_gates=False)

# output is a pure state, one of I|+>=|+>, X|+>=|+>, Y|+>, Z|+>=|->
print(psi)

[ 0.70710677+0.j -0.70710677+0.j]


In [10]:
# now run many samples and reconstruct the density matrix

p=0.25
n_sample = 250

gdc = GlobalDepolarizingChannel([0], p=p)
c = SuperCircuit([gdc])

states = []
for _ in range(n_sample):
    psi = simulate(c, '+', allow_sampling=True, remove_id_gates=False)
    states.append(psi)

# there is a function to sum up the pure states into a density matrix
rho_out = reconstruct_dm(states)

print(f"rho out from simulation of {n_sample} samples:")
print(np.round(rho_out, 4))

print()
print("expected rho_out:")
rho_exp = (1-p) * np.ones((2,2)) / 2 + p * np.eye(2) / 2
print(rho_exp)

rho out from simulation of 250 samples:
[[0.5  +0.j 0.376+0.j]
 [0.376+0.j 0.5  +0.j]]

expected rho_out:
[[0.5   0.375]
 [0.375 0.5  ]]


In [11]:
# now a channel with non-unitary gates that are sampled at run-time

adc = AmplitudeDampingChannel([0], gamma=0.25)
c = SuperCircuit([adc])

# we don't need to specify remove_id_gates=False here since the Kraus' are
# sampled at run-time (and none are identity)
psi = simulate(c, '1', allow_sampling=True)

# the output is either |1> or |0>, with probability depending on gamma.
print(psi)

[0.+0.j 1.+0.j]
